# Roof builder

Descrizione di una funzione che crea un tetto le cui falde hanno tutte la stessa pendenza a partire da un insieme di punti che descrivono la base del tetto. 
La funzione calcola i punti interni del tetto grazie alla funzione atan2 e poi realizza il modello 3d.

## Variables
- points: l'insieme dei punti base del tetto
- higherPoints: l'insieme dei punti alti del tetto
- alpha: l'angolo di pendenza delle falde del tetto
- space: la distanza tra i punti esterni e i punti interni del tetto


In [1]:
from larlib import *
from pyplasm import *

alpha = 60*PI/180
h = 2.0
points = [[0,0],[0,5],[7,5],[7,10],[12,10],[12,0]]

"""Funzione che crea un tetto a partire da un insieme di punti base del tetto"""
def roofBuilder(points,alpha):
    
    #Calcolo della distanza tra i punti esterni e i punti interni
    space = math.atan2(h,1/math.sin(alpha))

    xMin = xMax = yMin = yMax = 0
    for i in points:
        if i[0] > xMax:
            xMax = i[0]
        elif i[0] < xMin:
            xMin = i[0]
        elif i[1] > yMax:
            yMax = i[1]
        elif i[1] < yMin:
            yMin = i[1]
    
    #Calcolo dei punti alti del tetto
    higherPoints = []
    for i in range(0,len(points)):
        x = y = 0
        if points[i][0] == xMax:
            x = points[i][0]-space
        elif points[i][0] == xMin or (points[i-1][0] <= points[i][0] and  points[i+1][0] >= points[i][0]):
            x = points[i][0]+space
        
        if points[i][1] == yMin:
            y = points[i][1]+space
        elif points[i][1] == yMax or (points[i-1][1] <= points[i][1] and points[i+1][1] >= points[i][0]):
            y = points[i][1]-space
        point = [x,y]
        higherPoints.extend([point])
    
    #Crezione del soffitto del tetto
    ceiling = STRUCT(AA(POLYLINE)([[higherPoints[0],higherPoints[1],higherPoints[2],higherPoints[3],higherPoints[4],higherPoints[5],higherPoints[0]]]))
    ceiling = SOLIDIFY(ceiling)
    ceiling = STRUCT([T(3)(h),ceiling])
    
    for p in points:
        p.append(0)
    for hp in higherPoints:
        hp.append(h)
    newPoints = points+higherPoints
    
    #Creazione delle celle del tetto
    newCells = []
    for j in range(1,len(points)+1):
        if j != len(points):
            newCells.extend([[j,j+1,j+len(points),j+len(points)+1]])
        else:
            newCells.extend([[j,1,j+len(points),j+1]])
    
    #Creazione del modello
    roof = MKPOL([newPoints,newCells,None])
    roof = STRUCT([roof,ceiling])
    roof = OFFSET([.1,.1,.1])(roof)
    roof = COLOR(RED)(roof)
    
    
    VIEW(roof)
    return roof


roofBuilder(points,alpha)

Evaluating fenvs.py..
...fenvs.py imported in 0.009214 seconds


/usr/local/lib/python2.7/dist-packages/larlib-0.3.2-py2.7.egg/larlib/larstruct.py:233: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  self.body = [item for item in data if item != None]


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f7c49e3cdb0> >

# Results:
<img src="result1.png">
<img src="result2.png">
<img src="result3.png">